In [1]:
import numpy as np
import pandas as pd
from chromatography import *
from separation_utility import *
from torch import optim, tensor
import torch.nn as nn
import matplotlib.pyplot as plt
import time

%matplotlib qt

In [2]:
alists = []
alists.append(pd.read_csv(f'../data/GilarSample.csv'))
alists.append(pd.read_csv(f'../data/Peterpeptides.csv'))
alists.append(pd.read_csv(f'../data/Roca.csv'))
alists.append(pd.read_csv(f'../data/Peter32.csv'))
alists.append(pd.read_csv(f'../data/Eosin.csv'))
alists.append(pd.read_csv(f'../data/Alizarin.csv'))
alists.append(pd.read_csv(f'../data/Controlmix2.csv'))


In [3]:
def loss_field(exp, taus, N = 200):
    phis = np.linspace(0, 1, N)
    losses = np.zeros((N, N))
    j = 0
    for phi1 in phis:
        i = 0
        for phi2 in phis:
            exp.reset()
            exp.run_all([phi1, phi2], taus)
            losses[i, j] = exp.loss()
            i += 1
        j += 1
    X, Y = np.meshgrid(phis, phis)
    
    return X, Y, losses

def average_over_equal_intervals(arr, interval):
    return np.mean(arr.reshape(-1, interval), axis=1)

In [32]:

taus = [.25, .25, 10]

time_start_pp = time.perf_counter()
exp = ExperimentAnalytes(k0 = alists[0].k0.values, S = alists[0].S.values, h=0.001,run_time=10.0)
for i in range(20):
    _, _, mus, _ = reinforce_single_from_gen(
        alist=alists[0], 
        policy=pol, 
        delta_taus=taus, 
        num_episodes=2000, 
        sample_size=10, 
        lr=.05, 
        optim=torch.optim.SGD,
        lr_decay_factor=.5,
        lr_milestones=1000,
        print_every=100,
        baseline=0.55,
        max_norm=2.
    )
    exp.reset()
    exp.run_all(mus[-1,:], taus)
    
time_end_pp = time.perf_counter()
# time 523.31177600672

Loss: 0.7445388629086223, epoch: 100/2000
Loss: 1.0170569816298722, epoch: 200/2000
Loss: 0.68236809486308, epoch: 300/2000
Loss: 0.6799520671579715, epoch: 400/2000
Loss: 1.050534947262894, epoch: 500/2000
Loss: 0.780386113035355, epoch: 600/2000
Loss: 0.6988977648113541, epoch: 700/2000
Loss: 0.9433440217073807, epoch: 800/2000
Loss: 0.7154878737090586, epoch: 900/2000
Loss: 0.8969268508482584, epoch: 1000/2000
Loss: 0.674074411330176, epoch: 1100/2000
Loss: 0.7293916001050033, epoch: 1200/2000
Loss: 0.6881110535044157, epoch: 1300/2000
Loss: 0.7079102378072241, epoch: 1400/2000
Loss: 0.6875166655089711, epoch: 1500/2000
Loss: 0.6579160588269175, epoch: 1600/2000
Loss: 0.6907639568968329, epoch: 1700/2000
Loss: 0.6696473818667967, epoch: 1800/2000
Loss: 0.6788520841131311, epoch: 1900/2000
Loss: 0.5842320194847973, epoch: 2000/2000
Loss: 1.0699037513725527, epoch: 100/2000
Loss: 1.0196179767170555, epoch: 200/2000
Loss: 1.0057416851869987, epoch: 300/2000
Loss: 1.0137692256415531, ep

Loss: 0.583569068799548, epoch: 1500/2000
Loss: 0.5776934827012188, epoch: 1600/2000
Loss: 0.5722675845840844, epoch: 1700/2000
Loss: 0.5856392612137542, epoch: 1800/2000
Loss: 0.5486689668739066, epoch: 1900/2000
Loss: 0.5722237074163632, epoch: 2000/2000
Loss: 0.7614946405219418, epoch: 100/2000
Loss: 0.7003723012051141, epoch: 200/2000
Loss: 0.9818921602869899, epoch: 300/2000
Loss: 0.6830127977484455, epoch: 400/2000
Loss: 0.6068439418270161, epoch: 500/2000
Loss: 0.6689394424702512, epoch: 600/2000
Loss: 0.634258563702068, epoch: 700/2000
Loss: 0.5839292949536745, epoch: 800/2000
Loss: 0.5686169273840258, epoch: 900/2000
Loss: 0.5997516996493941, epoch: 1000/2000
Loss: 0.5533495394151489, epoch: 1100/2000
Loss: 0.5989707094485731, epoch: 1200/2000
Loss: 0.5976724911105992, epoch: 1300/2000
Loss: 0.5680820994304331, epoch: 1400/2000
Loss: 0.6035288551217378, epoch: 1500/2000
Loss: 0.5662350125541302, epoch: 1600/2000
Loss: 0.5494806450957767, epoch: 1700/2000
Loss: 0.58020189984165

Loss: 0.6099782446482876, epoch: 900/2000
Loss: 0.6099015180492293, epoch: 1000/2000
Loss: 0.5561390654747, epoch: 1100/2000
Loss: 0.5726311905564787, epoch: 1200/2000
Loss: 0.5815248616861004, epoch: 1300/2000
Loss: 0.5672312440890783, epoch: 1400/2000
Loss: 0.6109365015995468, epoch: 1500/2000
Loss: 0.5746759059936306, epoch: 1600/2000
Loss: 0.572588051518008, epoch: 1700/2000
Loss: 0.552369311427044, epoch: 1800/2000
Loss: 0.5681415402451359, epoch: 1900/2000
Loss: 0.5963962841447517, epoch: 2000/2000


In [36]:
8.514 - (time_end_pp - time_start_pp)/1200

7.665162367281697

In [29]:
plt.hist(losses_gen_pp, bins=50)
plt.title(f"Final Result Distribution for GenModel (PeterPeptides) Not in Training")
plt.xlabel("Loss")
plt.ylabel("Occurrences")

Text(0, 0.5, 'Occurrences')

In [30]:
plt.hist(losses_from_gen_pp, bins=50)
plt.title(f"Final Result Distribution for GenModel + FineTune(PeterPeptides) Not in Training")
plt.xlabel("Loss")
plt.ylabel("Occurrences")

Text(0, 0.5, 'Occurrences')

In [ ]:
#Gillar
taus = [.25, .25, 10]
pol = PolicyGeneral(
            phi = nn.Sequential(
                PermEqui2_max(2, 5),
                nn.ELU(inplace=True),
                PermEqui2_max(5, 5),
                nn.ELU(inplace=True),
                PermEqui2_max(5, 5),
                nn.ELU(inplace=True),
            ),
            rho = nn.Sequential(
                nn.Linear(5, 5),
                nn.ELU(inplace=True),
                nn.Linear(5, 5),
                nn.ELU(inplace=True),
                Rho(n_steps=len(taus), hidden=5, in_dim=5, sigma_max=.3, sigma_min=.01),
            )
        )
reinforce_gen(
    alists = alists, 
    policy = pol, 
    delta_taus = taus, 
    num_episodes = 40_000, 
    sample_size = 10,
    batch_size = 1, 
    lr = .05, 
    optim = lambda a, b: torch.optim.SGD(a, b),
    lr_decay_factor = 0.75,
    lr_milestones = 5000,
    print_every = 20_000,
    baseline = .55,
    max_norm = 1.5,
    max_rand_analytes = 30,
    min_rand_analytes = 15,
    rand_prob = 0.7
)

In [ ]:
plt.plot(average_over_equal_intervals(np.array(L8).mean(0), 500), label="L8")
plt.title("Loss (average of 500 random sets)")
plt.xlabel("Episode")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
i = 0
exp = ExperimentAnalytes(k0 = alists[i].k0.values, S = alists[i].S.values, h=0.001,run_time=10.0)
mu, sig = pol(torch.Tensor(alists[i][['S', 'lnk0']].values))
exp.run_all(mu.detach().numpy(), taus)

exp.print_analytes(title=f"Solvent Strength Program(Gen)\nLoss:{round(exp.loss(), 4)}", rc=(10,10), angle=40)

In [ ]:
L, B, mus, sigmas = reinforce_single_from_gen(
        alist=alists[0], 
        policy=pol, 
        delta_taus=taus, 
        num_episodes=2000, 
        sample_size=10, 
        lr=.05, 
        optim=torch.optim.SGD,
        lr_decay_factor=.5,
        lr_milestones=1000,
        print_every=100,
        baseline=0.55,
        max_norm=2.
    )

In [ ]:
exp.loss()

In [ ]:
i = 0
exp = ExperimentAnalytes(k0 = alists[i].k0.values, S = alists[i].S.values, h=0.001,run_time=10.0)
exp.run_all(mus[-1,:], taus)
exp.print_analytes(title=f"Solvent Strength Program(Iso)\nLoss:{round(exp.loss(), 4)}", rc=(10,10), angle=40)

In [ ]:
exp = ExperimentAnalytes(k0 = alists[i].k0.values, S = alists[i].S.values, h=0.001,run_time=10.0)
exp.run_all(B, taus)
exp.print_analytes(title=f"Best Solvent Strength Program\nLoss:{round(exp.loss(), 4)}", rc=(10,10), angle=40)

In [ ]:
plt.plot(mus)

In [ ]:
plt.plot(sigmas)

In [ ]:
pol = PolicyGeneral(
            phi = nn.Sequential(
                PermEqui2_max(2, 3),
                nn.ELU(inplace=True),
                PermEqui2_max(3, 3),
                nn.ELU(inplace=True),
                PermEqui2_max(3, 3),
                nn.ELU(inplace=True),
            ),
            rho = RhoTime(n_steps=3, hidden=5, in_dim=3, sigma_max=.3, sigma_min=.02)
        )
losses = reinforce_delta_tau_gen(
    alists = alists, 
    policy = pol,
    num_episodes = 10000, 
    batch_size = 10, 
    lr = .1, 
    optim = lambda a, b: torch.optim.SGD(a, b),
    lr_decay_factor= 0.75,
    lr_milestones=1000,
    print_every = 100,
    baseline = .55,
    max_norm = 1.2,
    max_rand_analytes = 35,
    min_rand_analytes = 18,
    rand_prob = 0.7
)

In [ ]:
plt.plot(np.linspace(0, 200000, 200),average_over_equal_intervals(losses[0], 1000))
plt.title("Loss [variable delta tau] (average of 1000 random sets)")
plt.xlabel("Episode")
plt.ylabel("Loss")
plt.show()

In [ ]:
i = 6
exp = ExperimentAnalytes(k0 = alists[i].k0.values, S = alists[i].S.values, h=0.001,run_time=10.0)
mu, _ = pol(torch.Tensor(alists[i][['S', 'lnk0']].values))
mu = mu.tolist()
mu.append(10.)
exp.run_all(mu[0:3], mu[3:])

exp.print_analytes(title=f"Solvent Strength Program\nLoss:{round(exp.loss(), 4)}", rc=(10,10), angle=40)

In [ ]:
L, B, mus, sigmas = reinforce_single_from_delta_tau_gen(
        alist=alists[i], 
        policy=pol,
        num_episodes=5000, 
        batch_size=10, 
        lr=.1, 
        optim=torch.optim.SGD,
        lr_decay_factor=.5,
        lr_milestones=500,
        print_every=500,
        baseline=0.65,
        max_norm=1.2
    )

In [ ]:
plt.plot(mus)

In [ ]:
plt.plot(L)

In [ ]:
exp = ExperimentAnalytes(k0 = alists[i].k0.values, S = alists[i].S.values, h=0.001,run_time=10.0)
mu = mus[-1].tolist()
mu.append(10.)
exp.run_all(mu[0:3], mu[3:])

exp.print_analytes(title=f"Solvent Strength Program\nLoss:{round(exp.loss(), 4)}", rc=(10,10), angle=40)

# NEW

In [ ]:
class PolicyGeneralISO(nn.Module):
    def __init__(self, 
            phi: nn.Module,
            rho: nn.Module
        ) -> None:
        """
        Constructor for PolicyTime torch Module.

        Parameters
        ----------
        phi: nn.Module
            The network that encodes the analyte set to a single 
            vector (embedding)
        rho: nn.Module
            The network that outputs the programe for separation
            returns mean and standard deviation of the action space

        Ex:
        For a 4 step solvent gradient programe the generalized policy 
        with 3 elements embedding for the analyte set and intermediate
        layers of 5 neurons.
        policy = PolicyGeneral(
            phi = nn.Sequential(
                PermEqui1_max(2, 5),
                nn.ELU(inplace=True),
                PermEqui1_max(5, 5),
                nn.ELU(inplace=True),
                PermEqui1_max(5, 3),
                nn.ELU(inplace=True),
            ),
            rho = Rho(4, 5, 3, .3, .05)
        )
        """
        super().__init__()

        self.phi = phi
        self.rho = rho
        
    def forward(self, x, y):
        phi_output = self.phi(x)
        sum_output = phi_output.sum(0, keepdim=True)
        sum_output = torch.cat([sum_output, y], 1)
        mu, sigma = self.rho(sum_output)
        return mu, sigma

#############################################################################
def reinforce_gen_iso(
        alists: Iterable[pd.DataFrame],
        policy: PolicyGeneral, 
        delta_taus: Iterable[float], 
        num_episodes: int = 1000, 
        sample_size: int = 10,
        batch_size : int = 10,
        lr: float = 1., 
        optim = torch.optim.SGD,
        lr_decay_factor: float = 1.,
        lr_milestones: Union[int, Iterable[int]] = 1000,
        rand_prob: float = .2,
        max_rand_analytes: int = 30,
        min_rand_analytes: int = 10,
        print_every: int = 100,
        baseline: float = 0.,
        max_norm: float = None,
        beta: float = .0,
        weights: list = [1., 1.]
    ):
    """
    Run Reinforcement Learning for a single set learning.

    alists: Iterable[pd.DataFrame]
        A list with pd.Dataframes for each dataset used to train on. 
    policy: PolicyGeneral
        The policy that learns the optimal values for the solvent
        strength program.
    delta_taus: Iterable[float]
        Iterable list with the points of solvent strength change.
        MUST be the same length as policy.n_steps
    num_episodes = 1000
        Number of learning steps.
    sample_size = 10
        Number of samples taken from the action distribution to perform 
        Expected loss for the distribution of actions.
    batch_size:
        Number of experiments to run in order to aproximate the true gradient.
    lr = 1.
        Learning rate.
    optim = torch.optim.SGD
        Optimizer that performs weight update using gradients.
        By defauld is Stochastic Gradient Descent.
    lr_decay_factor: float
        Learning rate decay factor used for the LRScheduler.
        lr is updated according to lr = lr ** lr_decay_factor.
    lr_milestones: Union[int, Iterable[int]]
        Milestone episode/s to update the learning rate.
        If it is int StepLR is used where lr is changed every lr_milestones.
        If it is a list of ints then at that specific episode the lr
        will be changed.
    rand_prob: float = .2
        The probability to draw a random subset from all the analytes.
        1 - rand_prob is the probability to use a "real" set (provided in
        alists).
    max_rand_analytes: int = 30
        The maximum number of analytes in the randomly drawn set.
    min_rand_analytes: int = 10
        The minimum number of analytes in the randomly drawn set.
    print_every = 100,
        Number of episodes to print the average loss on.
    weights = [1., 1.]
        Weigths of the errors to consider, first one is for the Placement Error,
        second one is for Overlap Error, By default both have the same wights.
    baseline = 0.
        Baseline value for the REINFORCE algorithm.
    max_norm = None
        Maximal value for the Neural Network Norm2.
    beta = .0
        Entropy Regularization term, is used for more exploration.
        By defauld is disabled.
    Returns
    -------
    (losses, best_program, mus, sigmas)
    losses: np.ndarray
        Expected loss of the action distribution over the whole learning
        process.
    """

    losses = []
    perfect_loss = []
    exps = []

    # Make ExperimentAnalytes object for the given analyte sets for time saving purpose
    for alist in alists:
        exps.append(ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, run_time=10.0))

    num_exps = len(alists)

    all_analytes = pd.concat(alists, sort=True)[['k0', 'S', 'lnk0']]

    # Optimizer
    optimizer = optim(policy.parameters(), lr)

    # LR sheduler
    if isinstance(lr_milestones, list) or isinstance(lr_milestones, np.ndarray):
        scheduler = MultiStepLR(optimizer, lr_milestones, gamma=lr_decay_factor)
    else:
        scheduler = StepLR(optimizer, lr_milestones, gamma=lr_decay_factor)

    J_batch = 0

    for n in range(num_episodes):
        # the set to use for the experiment.
        if random() < rand_prob:
            dataframe = all_analytes.sample(randint(min_rand_analytes, max_rand_analytes))
            input_data = torch.tensor(dataframe[['S', 'lnk0']].values, dtype=torch.float32)
            exp = ExperimentAnalytes(k0 = dataframe.k0.values, S = dataframe.S.values, h=0.001, run_time=10.0)

        else:
            # Choose a random set
            set_index = randint(0, num_exps - 1) 
            exp = exps[set_index]
            input_data = torch.tensor(alists[set_index][['S', 'lnk0']].values, dtype=torch.float32)
        
        expected_loss = 10
        for phi in np.linspace(0, 1, 100):
            exp.reset()
            exp.step(phi, 1.)
            if exp.loss() < expected_loss:
                phi_iso = phi
                expected_loss = exp.loss()    
        
        # compute distribution parameters (Normal)
        mu, sigma = policy.forward(input_data, torch.tensor([[phi_iso]]))

        # Sample some values from the actions distributions
        programs = sample(mu, sigma, sample_size)
        
        # Fit the sampled data to the constraint [0,1]
        constr_programs = programs.clone()
        constr_programs[constr_programs > 1] = 1
        constr_programs[constr_programs < 0] = 0
        
        J = 0
        expected_loss = 0
        for i in range(sample_size):
            exp.reset()            
            exp.run_all(constr_programs[i].data.numpy(), delta_taus)

            error = exp.loss(weights)
            expected_loss += error
            log_prob_ = log_prob(programs[i], mu, sigma)
            J += (error - baseline) * log_prob_ - beta * torch.exp(log_prob_) * log_prob_
        
        losses.append(expected_loss/sample_size)
        perfect_loss.append(exp.perfect_loss(weights))
        if (n + 1) % print_every == 0:
            print(f"Loss: {losses[-1]}, epoch: {n+1}/{num_episodes}")

        J_batch += J/sample_size
        if (i + 1) % batch_size == 0:
            J_batch /= batch_size
            optimizer.zero_grad()
            # Calculate gradients
            J_batch.backward()

            if max_norm:
                torch.nn.utils.clip_grad_norm_(policy.parameters(), max_norm)

            # Apply gradients
            optimizer.step()

            # learning rate decay
            scheduler.step()

            J_batch = 0
        
    return np.array(losses), np.array(perfect_loss)

In [ ]:
taus = [.25, .25, 10]
L5_b1_iso = []
PL5_b1_iso = []
for i in range(5):
    pol = PolicyGeneralISO(
                phi = nn.Sequential(
                    PermEqui2_max(2, 5),
                    nn.ELU(inplace=True),
                    PermEqui2_max(5, 5),
                    nn.ELU(inplace=True),
                    PermEqui2_max(5, 5),
                    nn.ELU(inplace=True),
                ),
                rho = nn.Sequential(
                    nn.Linear(6, 6),
                    nn.ELU(inplace=True),
                    nn.Linear(6, 6),
                    nn.ELU(inplace=True),
                    Rho(n_steps=len(taus), hidden=6, in_dim=6, sigma_max=.3, sigma_min=.01),
                )
            )
    l, p = reinforce_gen_iso(
        alists = alists, 
        policy = pol, 
        delta_taus = taus, 
        num_episodes = 20_000, 
        sample_size = 10,
        batch_size = 1, 
        lr = .05, 
        optim = lambda a, b: torch.optim.SGD(a, b),
        lr_decay_factor = 0.75,
        lr_milestones = 5000,
        print_every = 5000,
        baseline = .55,
        max_norm = 1.7,
        max_rand_analytes = 30,
        min_rand_analytes = 10,
        rand_prob = 0.7
    )
    L5_b1_iso.append(l)
    PL5_b1_iso.append(p)

In [ ]:
plt.figure()
plt.plot((np.array(L5_b1) - np.array(PL5_b1)).reshape((10, 200, 100)).mean(2).T)
plt.ylim((0.3, 1))
plt.title("Simple batch one small")

In [ ]:
plt.figure()
#plt.plot((np.array(L5_b1_iso) - np.array(PL5_b1_iso)).mean(0).reshape(-1, 100).mean(1), label = "ISO")
plt.plot((np.array(L5_b1) - np.array(PL5_b1)).mean(0).reshape(-1, 100).mean(1), label = "Small NN")
#plt.plot((np.array(L5_b1_big) - np.array(PL5_b1_big)).mean(0).reshape(-1, 100).mean(1), label = "Big NN")
plt.title("Small rho NN vs Big rho NN")
plt.legend()

In [ ]:
(np.array(L5_b1) - np.array(PL5_b1)).mean(0).reshape(-1, 100).mean(1)